# Lectura de Base de Datos y Creación de archivos CSV

<font size=5>Reducción de Emisiones Contaminantes por el Uso de Biocombustibles en Transporte de Cargas y Pasajeros</font>

<font size=4>Mentoria DiploDatos FAMAF </font>

In [2]:
pip freeze

appdirs==1.4.4
attrs==19.3.0
backcall==0.2.0
bleach==3.1.5
brotlipy==0.7.0
certifi==2020.6.20
cffi==1.14.0
chardet==3.0.4
click==6.7
click-plugins==1.1.1
cligj==0.5.0
colorama==0.4.3
cryptography==2.9.2
cycler==0.10.0
decorator==4.4.2
defusedxml==0.6.0
descartes==1.1.0
elevation==1.0.6
entrypoints==0.3
fasteners==0.15
Fiona==1.8.4
future==0.18.2
GDAL==2.3.3
geopandas==0.6.1
idna==2.10
importlib-metadata==1.7.0
ipykernel==5.3.0
ipython==7.16.1
ipython-genutils==0.2.0
jedi==0.17.1
Jinja2==2.11.2
json5==0.9.5
jsonschema==3.2.0
jupyter-client==6.1.3
jupyter-core==4.6.3
jupyterlab==2.1.5
jupyterlab-server==1.1.5
kiwisolver==1.2.0
MarkupSafe==1.1.1
matplotlib==3.2.2
memory-profiler==0.57.0
mistune==0.8.4
mkl-fft==1.1.0
mkl-random==1.1.1
mkl-service==2.3.0
monotonic==1.5
munch==2.5.0
nbconvert==5.6.1
nbformat==5.0.7
notebook==6.0.3
numpy==1.18.5
packaging==20.4
pandas==1.0.5
pandocfilters==1.4.2
parso==0.7.0
patool==1.12
pickleshare==0.7.5
prometheus-client==0.8.0
prompt-toolkit==3.0.5
psutil

In [1]:
import pandas as pd
import numpy as np
#pip install pymysql
from sqlalchemy import create_engine, inspect
import warnings

In [2]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 50)

## Datos propios
_Datasets :_
- Usuarios: registra datos de los usuarios de las bombas
- Vehiculos: registra datos de los vehículos 
- Productos: registra datos de los tipos de combustibles
- Equipos: registra datos de los equipos IoT
- Tanques: registra datos de los tanques de almacenamiento
- Bombas: registra datos de las bombas de suministro de combustible
- Transacciones (mensuales, desde el 2018): registro de los suministros de combustible por cada bomba a cada vehículo
- Historial (mensuales, desde el 2018): registro de los inventarios mensuales de los tanques de combustibles

In [3]:
# pip install memory_profiler
%load_ext memory_profiler

In [4]:
# Datos de conexión a la base de datos
host = "localhost"                   # host='localhost'
user = "root"                        # usuario de la base de datos ('root' by default)
password = ""                        # contraseña, sino tiene: ""
database = "nombre_db"               # nombre asignado a la base de datos
port = 3306                          # puerto de conexión remota

# Conexión a la base de datos (para MySQL)
from sqlalchemy import create_engine

# Crear motor de conexión sqlalchemy
path_conexion = f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}'
conexion = create_engine(path_conexion)

# Listado de todas las tablas de la base de datos
inspector = inspect(conexion)
lista_tablas = inspector.get_table_names()
print(lista_tablas)

['fs_asignacion_producto', 'fs_bombas', 'fs_equipo', 'fs_tanques', 'fs_usuarios_fuelsentry', 'fs_vehiculos', 'sis_historial_2018_1', 'sis_historial_2018_10', 'sis_historial_2018_11', 'sis_historial_2018_12', 'sis_historial_2018_2', 'sis_historial_2018_3', 'sis_historial_2018_4', 'sis_historial_2018_5', 'sis_historial_2018_6', 'sis_historial_2018_7', 'sis_historial_2018_8', 'sis_historial_2018_9', 'sis_historial_2019_1', 'sis_historial_2019_10', 'sis_historial_2019_11', 'sis_historial_2019_12', 'sis_historial_2019_2', 'sis_historial_2019_3', 'sis_historial_2019_4', 'sis_historial_2019_5', 'sis_historial_2019_6', 'sis_historial_2019_7', 'sis_historial_2019_8', 'sis_historial_2019_9', 'sis_historial_2020_1', 'sis_historial_2020_2', 'sis_historial_2020_3', 'sis_historial_2020_4', 'sis_historial_2020_5', 'sis_transa_2018_1', 'sis_transa_2018_10', 'sis_transa_2018_11', 'sis_transa_2018_12', 'sis_transa_2018_2', 'sis_transa_2018_3', 'sis_transa_2018_4', 'sis_transa_2018_5', 'sis_transa_2018_6

In [5]:
# Run: Script de funciones a utilizar
%run functions.py

In [6]:
%%time
%memit
dataframes = [pd.read_sql_query(f"SELECT * FROM {tabla};", conexion) for tabla in lista_tablas[:6]]

for idx, table_name in enumerate(lista_tablas[:6]):
    vars()[table_name] = dataframes[idx]
    print(f'Dataframe {idx+1}: {table_name}')

peak memory: 128.93 MiB, increment: 0.31 MiB
Dataframe 1: fs_asignacion_producto
Dataframe 2: fs_bombas
Dataframe 3: fs_equipo
Dataframe 4: fs_tanques
Dataframe 5: fs_usuarios_fuelsentry
Dataframe 6: fs_vehiculos
Wall time: 14.1 s


### Dataset de asignación de productos:

Contiene información relacionada con los productos de combustible.

Descripción de las columnas:
- 'id_equipo': ID del dispositivo IoT
- 'producto': código del producto
- 'nombre_producto': nombre del producto
- 'codigo': 
- 'precio_litro': precio del producto (en $/l)
- 'coef_var_vol': coeficiente de variación del volumen del producto combustible (en g/cm3/°C)
- 'density': densidad del producto (en g/cm3)

Densidad (ASTM D 4052): densidad relativa del combustible medido a la temperatura estándar de 15 °C.

In [7]:
detalles_dataframe(fs_asignacion_producto)

Dimensión del dataset (filas, columnas): (2672, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2672 entries, 0 to 2671
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_equipo        2672 non-null   int64  
 1   producto         2672 non-null   object 
 2   nombre_producto  2672 non-null   object 
 3   codigo           2526 non-null   object 
 4   precio_litro     2672 non-null   float64
 5   coef_var_vol     2672 non-null   float64
 6   density          249 non-null    float64
dtypes: float64(3), int64(1), object(3)
memory usage: 146.2+ KB
None



,id_equipo,producto,nombre_producto,codigo,precio_litro,coef_var_vol,density
0,333333,0,gas oil,gas oil,7.5,0.001,NaN
1,333333,1,Product 2,Product 2,0.0,0.001,NaN
2,333333,2,Product 3,Product 3,0.0,0.001,NaN
3,333333,3,Product 4,Product 4,0.0,0.001,NaN
4,1,0,Diesel,Diesel,1.0,0.001,NaN


### Dataset de bombas:

Contiene información de las bombas de suministro de combustible.

--> Un punto de suministro del combustible posee uno o varios equipos (dispositivo IoT) 

Descripción de las columnas:
- 'id_bomba': ID de la bomba
- 'id_equipo': ID del dispositivo IoT
- 'bomba': 
- 'producto': tipo de combustible que suministra la bomba
- 'id_tanque': ID del tanque
- 'totalizador': litros suministrados a la fecha de reinicio
- 'fecha_reinicio': fecha de reinicio de la bomba
- 'pulsos_litro': litros que suministra la bomba (por pulso)
- 'tiempo_interrump': tiempo de interrupción del suministro de combustible de la bomba (en segundos)
- 'habilitacion': 
- 'rampa_de_parada'

In [8]:
detalles_dataframe(fs_bombas)

Dimensión del dataset (filas, columnas): (1438, 11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1438 entries, 0 to 1437
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id_bomba          1438 non-null   int64 
 1   id_equipo         1438 non-null   int64 
 2   bomba             1438 non-null   object
 3   producto          1438 non-null   object
 4   id_tanque         1438 non-null   int64 
 5   totalizador       1438 non-null   object
 6   fecha_reinicio    1438 non-null   object
 7   pulsos_litro      1438 non-null   object
 8   tiempo_interrump  1438 non-null   object
 9   habilitacion      1438 non-null   object
 10  rampa_de_parada   1438 non-null   object
dtypes: int64(3), object(8)
memory usage: 123.7+ KB
None



,id_bomba,id_equipo,bomba,producto,id_tanque,totalizador,fecha_reinicio,pulsos_litro,tiempo_interrump,habilitacion,rampa_de_parada
0,1,333333,01,0,1,0031731.441,2012-07-19,0000,2.0,1,0000
1,2,333333,02,0,1,1160601.525,2012-07-19,0000,2.0,1,0000
2,11417,770,02,1,101856,0000000.000,2018-07-05,0026.3158,020,1,0000
3,5,444444,01,2,4,0651995.027,2010-12-01,35.0,888,1,0000
4,6,444444,02,2,4,0005232.793,2010-12-01,35.5,2.0,1,0000


### Dataset de tanques:

Contiene información de los tanques de combustible.

--> Una empresa puede tener más de un tanque

--> Varios tanques pueden estar conectado a un mismo equipo

--> Un tanque posee una o varias bombas

Descripción de las columnas:
- 'id_tanque': ID del tanque
- 'id_equipo': ID del dispositivo IoT 
- 'tanque': tipo de tanque
- 'producto': producto almacenado en el tanque 
- 'capacidad': capacidad del tanque en litros
- 'log_interval': intervalo de tiempo en el cual se registra la medición del volumen del contenido del tanque --> historial del volumen del tanque
- 'nivel_alarma': nivel de contenido del tanque para disparar alarma

In [9]:
detalles_dataframe(fs_tanques)

Dimensión del dataset (filas, columnas): (1561, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1561 entries, 0 to 1560
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_tanque     1561 non-null   int64 
 1   id_equipo     1561 non-null   int64 
 2   tanque        1561 non-null   object
 3   producto      1561 non-null   object
 4   capacidad     1561 non-null   object
 5   log_interval  1561 non-null   object
 6   nivel_alarma  1561 non-null   object
dtypes: int64(2), object(5)
memory usage: 85.5+ KB
None



,id_tanque,id_equipo,tanque,producto,capacidad,log_interval,nivel_alarma
0,101525,669,B,1,0010000.1,0030,0
1,101833,764,5,1,0008000.0,0030,000003200
2,101496,662,1,0,0050000.0,0030,0
3,101495,661,1,0,0025000.0,0030,0
4,101522,669,8,1,0010000.1,0030,0


### Dataset de usuarios:

Contiene información de los usuarios de las bombas de combustible.

Descripción de las columnas:
- 'id_usuario_fuelsentry': ID de registro del usuario
- 'id_equipo':  ID del dispositivo IoT
- 'usuario_fuelsentry': código del usuario de la bomba
- 'departamento': 
- 'codigo': 
- 'totalizador': litros totales suministrados por el usuario de la bomba
- 'cargas_totales': número total de cargas sumnistradas por el usuario

In [10]:
detalles_dataframe(fs_usuarios_fuelsentry)

Dimensión del dataset (filas, columnas): (13623, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13623 entries, 0 to 13622
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id_usuario_fuelsentry  13623 non-null  int64 
 1   id_equipo              13623 non-null  int64 
 2   usuario_fuelsentry     13623 non-null  object
 3   departamento           13623 non-null  object
 4   codigo                 13623 non-null  object
 5   totalizador            13623 non-null  object
 6   cargas_totales         13623 non-null  object
dtypes: int64(2), object(5)
memory usage: 745.1+ KB
None



,id_usuario_fuelsentry,id_equipo,usuario_fuelsentry,departamento,codigo,totalizador,cargas_totales
0,2,333333,0001,0000,0000,00323.152,001
1,3,333333,1315,0001,2012,01655.590,009
2,4,333333,1312,0001,4468,01433.429,006
3,5,333333,1298,0001,2609,00412.834,002
4,6,333333,0818,0001,1978,00713.023,003


### Dataset de vehículos:

In [11]:
detalles_dataframe(fs_vehiculos)

Dimensión del dataset (filas, columnas): (50627, 13)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50627 entries, 0 to 50626
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id_vehiculo       50627 non-null  int64 
 1   id_equipo         50627 non-null  int64 
 2   vehiculo          50627 non-null  object
 3   departamento      50627 non-null  object
 4   limite            50627 non-null  object
 5   odometro_inicio   50627 non-null  object
 6   odometro_fin      50627 non-null  object
 7   cargas_max_dia    50627 non-null  object
 8   autorizacion      50627 non-null  object
 9   cantidad_total    50627 non-null  object
 10  cargas_hoy        50627 non-null  object
 11  cargas_hasta_hoy  50627 non-null  object
 12  ultima_fecha      50627 non-null  object
dtypes: int64(2), object(11)
memory usage: 5.0+ MB
None



,id_vehiculo,id_equipo,vehiculo,departamento,limite,odometro_inicio,odometro_fin,cargas_max_dia,autorizacion,cantidad_total,cargas_hoy,cargas_hasta_hoy,ultima_fecha
0,2,333333,MDAwMDAx,0001,9,193958,193958,99,0,00000.000,01,000,2018-12-20
1,3,333333,MDAwMjM5,0001,9,079345,079345,99,0,00000.000,02,000,2018-12-20
2,4,333333,MDAwMjcw,0001,9,842409,842409,99,0,00000.000,01,000,2018-12-20
3,5,333333,MDAwMjUy,0001,9,287596,287596,99,0,00000.000,02,000,2018-12-20
4,6,333333,MDAwMjM1,0001,9,235878,235878,99,0,00000.000,01,000,2018-12-20


In [12]:
# Escribiendo data a archivos CSV
fs_asignacion_producto.to_csv('data/fs_asignacion_producto.csv', index=False)
fs_bombas.to_csv('data/fs_bombas.csv', index=False)
fs_equipo.to_csv('data/fs_equipo.csv', index=False)
fs_tanques.to_csv('data/fs_tanques.csv', index=False)
fs_usuarios_fuelsentry.to_csv('data/fs_usuarios_fuelsentry.csv', index=False)
fs_vehiculos.to_csv('data/fs_vehiculos.csv', index=False)

### Dataset de transacciones:

In [13]:
%%time
%memit
tablas_T_S1_2018 = ['sis_transa_2018_1', 'sis_transa_2018_2', 'sis_transa_2018_3', 'sis_transa_2018_4', 'sis_transa_2018_5', 'sis_transa_2018_6']
sis_transa_S1_2018 = dataframe_desde_querys(tablas_T_S1_2018, conexion)
detalles_dataframe(sis_transa_S1_2018)

peak memory: 180.70 MiB, increment: 0.04 MiB
Dimensión del dataset (filas, columnas): (505474, 31)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505474 entries, 0 to 505473
Data columns (total 31 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   id_transaccion  505474 non-null  int64          
 1   id_vehiculo     505474 non-null  int64          
 2   id_usuario      505474 non-null  int64          
 3   id_equipo       505474 non-null  int64          
 4   producto        505474 non-null  object         
 5   id_bomba        505474 non-null  int64          
 6   id_tanque       505474 non-null  int64          
 7   departamento    505474 non-null  object         
 8   fecha           505474 non-null  object         
 9   hora            505474 non-null  timedelta64[ns]
 10  cantidad        505474 non-null  object         
 11  acum_vehiculo   505474 non-null  object         
 12  acum_usuario    505474 non-n

,id_transaccion,id_vehiculo,id_usuario,id_equipo,producto,id_bomba,id_tanque,departamento,fecha,hora,cantidad,acum_vehiculo,acum_usuario,odometro,km_transaccion,codigo_error,valor,campovar1,campovar2,baja,fecha_stop,hora_stop,volume_start,volume_stop,temp_start,temp_stop,local_price,geo_latitud,geo_longitud,geo_status,veh_efficiency
0,6974,39920,1045,139,1,174,134,0000,2018-01-01,08:27:00,115.972,30665.8,99999.4,000012,0000,BF,0.0,0000,None,0,None,NaT,,,,,,,,,
1,6975,39819,1041,139,1,174,134,0000,2018-01-01,13:56:00,127.907,04723.7,79600.4,435885,0531,BF,0.0,0000,None,0,None,NaT,,,,,,,,,
2,6976,39931,1055,139,1,174,134,0000,2018-01-01,16:54:00,65.571,04324.0,42312.1,000000,0000,BF,0.0,0000,None,0,None,NaT,,,,,,,,,
3,6977,39922,1047,139,1,174,134,0000,2018-01-01,20:38:00,71.701,02506.3,14088.7,000000,0000,BF,0.0,0000,None,0,None,NaT,,,,,,,,,
4,5250,41504,436,111,1,100,81,0001,2018-01-01,00:23:00,360.018,0000124,0000296,464294,0000,C1,0.0,0000,None,0,None,NaT,,,,,,,,,


In [14]:
%%time
%memit
tablas_T_S2_2018 = ['sis_transa_2018_7', 'sis_transa_2018_8', 'sis_transa_2018_9', 'sis_transa_2018_10', 'sis_transa_2018_11', 'sis_transa_2018_12']
sis_transa_S2_2018 = dataframe_desde_querys(tablas_T_S2_2018, conexion)
detalles_dataframe(sis_transa_S2_2018)

peak memory: 692.75 MiB, increment: 0.00 MiB
Dimensión del dataset (filas, columnas): (573027, 31)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573027 entries, 0 to 573026
Data columns (total 31 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   id_transaccion  573027 non-null  int64          
 1   id_vehiculo     573027 non-null  int64          
 2   id_usuario      573027 non-null  int64          
 3   id_equipo       573027 non-null  int64          
 4   producto        573027 non-null  object         
 5   id_bomba        573027 non-null  int64          
 6   id_tanque       573027 non-null  int64          
 7   departamento    573027 non-null  object         
 8   fecha           573027 non-null  object         
 9   hora            573027 non-null  timedelta64[ns]
 10  cantidad        573027 non-null  object         
 11  acum_vehiculo   573027 non-null  object         
 12  acum_usuario    573027 non-n

,id_transaccion,id_vehiculo,id_usuario,id_equipo,producto,id_bomba,id_tanque,departamento,fecha,hora,cantidad,acum_vehiculo,acum_usuario,odometro,km_transaccion,codigo_error,valor,campovar1,campovar2,baja,fecha_stop,hora_stop,volume_start,volume_stop,temp_start,temp_stop,local_price,geo_latitud,geo_longitud,geo_status,veh_efficiency
0,1626,51118,0,346,0,545,735,0003,2018-07-01,00:07:00,20.288,14082.7,00000.0,000000,0000,C1,0.000,0000,None,0,0000-00-00,00:00:00,,,,,,,,,
1,8944,56832,5361,323,0,847,1023,0001,2018-07-01,00:01:00,12.812,00000.0,00000.0,047820,0038,C1,0.000,0000,None,0,0000-00-00,00:00:00,,,,,,,,,
2,9803,0,0,477,1,0,1012,0000,2018-07-01,00:11:24,6.612,0000000,0000000,000000,0000,ES,182.557,0000,None,0,2018-07-01,00:11:24,000000.000,000000.000,000.0,000.0,00000000,000.000000,0000.000000,N,0000.00
3,9804,0,0,477,1,0,1012,0000,2018-07-01,00:19:05,6.609,0000000,0000000,000000,0000,ES,182.474,0000,None,0,2018-07-01,00:19:05,000000.000,000000.000,000.0,000.0,00000000,000.000000,0000.000000,N,0000.00
4,9805,0,0,477,1,829,1011,0000,2018-07-01,00:21:26,39.621,0000000,0000000,000000,0000,ES,1227.850,0000,None,0,2018-07-01,00:21:26,000000.000,004272.200,000.0,014.5,00000000,000.000000,0000.000000,N,0000.00


In [15]:
%%time
%memit
tablas_T_S1_2019 = ['sis_transa_2019_1', 'sis_transa_2019_2', 'sis_transa_2019_3', 'sis_transa_2019_4', 'sis_transa_2019_5', 'sis_transa_2019_6']
sis_transa_S1_2019 = dataframe_desde_querys(tablas_T_S1_2019, conexion)
detalles_dataframe(sis_transa_S1_2019)

peak memory: 1319.68 MiB, increment: 0.00 MiB
Dimensión del dataset (filas, columnas): (614117, 31)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614117 entries, 0 to 614116
Data columns (total 31 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   id_transaccion  614117 non-null  int64          
 1   id_vehiculo     614117 non-null  int64          
 2   id_usuario      614117 non-null  int64          
 3   id_equipo       614117 non-null  int64          
 4   producto        614117 non-null  object         
 5   id_bomba        614117 non-null  int64          
 6   id_tanque       614117 non-null  int64          
 7   departamento    614117 non-null  object         
 8   fecha           614117 non-null  object         
 9   hora            614117 non-null  timedelta64[ns]
 10  cantidad        614117 non-null  object         
 11  acum_vehiculo   614117 non-null  object         
 12  acum_usuario    614117 non-

,id_transaccion,id_vehiculo,id_usuario,id_equipo,producto,id_bomba,id_tanque,departamento,fecha,hora,cantidad,acum_vehiculo,acum_usuario,odometro,km_transaccion,codigo_error,valor,campovar1,campovar2,baja,fecha_stop,hora_stop,volume_start,volume_stop,temp_start,temp_stop,local_price,geo_latitud,geo_longitud,geo_status,veh_efficiency
0,569,59995,5997,453,0,915,1133,0000,2019-01-01,00:02:19,41.598,0000000,0000000,000000,0000,C1,0.000,0000,None,0,2019-01-01,00:03:29,037610.766,037571.952,027.0,027.0,00000000,000.000000,0000.000000,N,0000.00
1,570,59983,5997,453,0,915,1133,0000,2019-01-01,00:07:32,18.781,0000000,0000000,000000,0000,C1,0.000,0000,None,0,2019-01-01,00:08:35,037565.483,037533.138,027.0,027.0,00000000,000.000000,0000.000000,N,0000.00
2,1999,59719,5905,477,0,832,1010,0000,2019-01-01,00:52:33,20.121,0000000,0000000,000000,0000,ES,623.550,0000,None,0,2019-01-01,00:52:33,000000.000,002804.898,000.0,024.0,00000000,000.000000,0000.000000,N,0000.00
3,913,61444,6191,639,1,10909,101093,0001,2019-01-01,00:34:09,95.493,0000155,0004030,026054,0757,C4,0.000,0001,None,0,2019-01-01,00:35:43,025349.984,025272.650,025.2,025.2,00000000,000.000000,0000.000000,N,0012.61
4,9853,42981,6190,210,0,372,367,0001,2019-01-01,01:01:00,124.875,0000006,0001615,668928,0507,C4,124.875,0001,None,0,0000-00-00,00:00:00,,,,,,,,,


In [16]:
%%time
%memit
tablas_T_S2_2019 = ['sis_transa_2019_7', 'sis_transa_2019_8', 'sis_transa_2019_9', 'sis_transa_2019_10', 'sis_transa_2019_11', 'sis_transa_2019_12']
sis_transa_S2_2019 = dataframe_desde_querys(tablas_T_S2_2019, conexion)
detalles_dataframe(sis_transa_S2_2019)

peak memory: 2013.77 MiB, increment: 0.00 MiB
Dimensión del dataset (filas, columnas): (679073, 31)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679073 entries, 0 to 679072
Data columns (total 31 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   id_transaccion  679073 non-null  int64          
 1   id_vehiculo     679073 non-null  int64          
 2   id_usuario      679073 non-null  int64          
 3   id_equipo       679073 non-null  int64          
 4   producto        679073 non-null  object         
 5   id_bomba        679073 non-null  int64          
 6   id_tanque       679073 non-null  int64          
 7   departamento    679073 non-null  object         
 8   fecha           679073 non-null  object         
 9   hora            679073 non-null  timedelta64[ns]
 10  cantidad        679073 non-null  object         
 11  acum_vehiculo   679073 non-null  object         
 12  acum_usuario    679073 non-

,id_transaccion,id_vehiculo,id_usuario,id_equipo,producto,id_bomba,id_tanque,departamento,fecha,hora,cantidad,acum_vehiculo,acum_usuario,odometro,km_transaccion,codigo_error,valor,campovar1,campovar2,baja,fecha_stop,hora_stop,volume_start,volume_stop,temp_start,temp_stop,local_price,geo_latitud,geo_longitud,geo_status,veh_efficiency
0,4303,68258,6062,438,2,929,1155,0001,2019-07-01,00:10:24,490.035,0000000,0000000,000000,0000,C1,0.00,0000,None,0,2019-07-01,00:19:40,-1.000,-1.000,000.0,000.0,00000000,,,,
1,8383,59719,5905,477,0,832,1010,0000,2019-07-01,00:10:48,41.344,0000000,0000000,000000,0000,ES,1281.25,0000,None,0,2019-07-01,00:10:48,000000.000,003963.386,000.0,016.5,00000000,000.000000,0000.000000,N,0000.00
2,8143,44856,5897,278,0,396,510,0000,2019-07-01,00:09:00,110.338,0000312,0000341,061440,0000,C1,2592.94,0000,None,0,0000-00-00,00:00:00,,,,,,,,,
3,6831,45552,1796,276,2,429,570,0001,2019-07-01,00:06:00,195.537,0000490,0000211,622655,0616,C1,0.00,0000,None,0,0000-00-00,00:00:00,,,,,,,,,
4,4647,49805,0,213,0,441,381,0001,2019-07-01,00:25:00,288.741,11484.8,00000.0,071052,0930,C1,0.00,,None,0,0000-00-00,00:00:00,,,,,,,,,


In [17]:
%%time
%memit
tablas_T_S1_2020 = ['sis_transa_2020_1', 'sis_transa_2020_2', 'sis_transa_2020_3', 'sis_transa_2020_4', 'sis_transa_2020_5']
sis_transa_S1_2020 = dataframe_desde_querys(tablas_T_S1_2020, conexion)
detalles_dataframe(sis_transa_S1_2020)

peak memory: 2803.47 MiB, increment: 0.00 MiB
Dimensión del dataset (filas, columnas): (448758, 31)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448758 entries, 0 to 448757
Data columns (total 31 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   id_transaccion  448758 non-null  int64          
 1   id_vehiculo     448758 non-null  int64          
 2   id_usuario      448758 non-null  int64          
 3   id_equipo       448758 non-null  int64          
 4   producto        448758 non-null  object         
 5   id_bomba        448758 non-null  int64          
 6   id_tanque       448758 non-null  int64          
 7   departamento    448758 non-null  object         
 8   fecha           448758 non-null  object         
 9   hora            448758 non-null  timedelta64[ns]
 10  cantidad        448758 non-null  object         
 11  acum_vehiculo   448758 non-null  object         
 12  acum_usuario    448758 non-

,id_transaccion,id_vehiculo,id_usuario,id_equipo,producto,id_bomba,id_tanque,departamento,fecha,hora,cantidad,acum_vehiculo,acum_usuario,odometro,km_transaccion,codigo_error,valor,campovar1,campovar2,baja,fecha_stop,hora_stop,volume_start,volume_stop,temp_start,temp_stop,local_price,geo_latitud,geo_longitud,geo_status,veh_efficiency
0,5402,59978,7467,453,0,915,1133,0000,2020-01-01,00:02:55,4.24,0000000,0000000,000000,0000,C1,227.052,0000,None,0,2020-01-01,00:03:20,005563.340,005563.340,029.0,029.0,00000000,000.000000,0000.000000,N,0000.00
1,5403,72368,7467,453,1,916,101175,0001,2020-01-01,00:07:06,69.011,0000000,0000000,000000,0000,C1,4220.020,0000,None,0,2020-01-01,00:08:19,006252.474,006231.066,030.0,030.0,00000000,000.000000,0000.000000,N,0000.00
2,5404,59972,7467,453,0,915,1133,0000,2020-01-01,00:12:36,81.688,0000000,0000000,000000,0000,C1,4374.390,0000,None,0,2020-01-01,00:14:15,005563.340,005530.995,029.0,029.0,00000000,000.000000,0000.000000,N,0000.00
3,5405,59957,7467,453,0,915,1133,0000,2020-01-01,00:15:49,26.128,0000000,0000000,000000,0000,C1,1399.150,0000,None,0,2020-01-01,00:16:59,005472.774,005472.774,029.0,029.0,00000000,000.000000,0000.000000,N,0000.00
4,5406,59986,7467,453,0,915,1133,0000,2020-01-01,00:18:45,74.35,0000000,0000000,000000,0000,C1,3981.440,0000,None,0,2020-01-01,00:20:21,005453.367,005382.208,029.0,029.0,00000000,000.000000,0000.000000,N,0000.00


In [18]:
# Escribiendo data a archivos CSV
sis_transa_S1_2018.to_csv('data/sis_transa_S1_2018.csv', index=False)
sis_transa_S2_2018.to_csv('data/sis_transa_S2_2018.csv', index=False)
sis_transa_S1_2019.to_csv('data/sis_transa_S1_2019.csv', index=False)
sis_transa_S2_2019.to_csv('data/sis_transa_S2_2019.csv', index=False)
sis_transa_S1_2020.to_csv('data/sis_transa_S1_2020.csv', index=False)

### Dataset de historial:

In [19]:
%%time
%memit
tablas_H_S1_2018 = ['sis_historial_2018_1', 'sis_historial_2018_2', 'sis_historial_2018_3', 'sis_historial_2018_4', 'sis_historial_2018_5', 'sis_historial_2018_6']
sis_historial_S1_2018 = query_union_to_dataframe_with_chunk(tablas_H_S1_2018, conexion)
detalles_dataframe(sis_historial_S1_2018)

peak memory: 3307.11 MiB, increment: 0.00 MiB
Dimensión del dataset (filas, columnas): (2977406, 20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2977406 entries, 0 to 2977405
Data columns (total 20 columns):
 #   Column            Dtype          
---  ------            -----          
 0   id                int64          
 1   id_equipo         int64          
 2   id_tanque         int64          
 3   fecha             object         
 4   hora              timedelta64[ns]
 5   volumen           object         
 6   temperatura       object         
 7   codigo            object         
 8   baja              int64          
 9   fuel_level_dmm    object         
 10  water_level_dmm   object         
 11  water_volume_lts  object         
 12  producto          object         
 13  geo_latitude      object         
 14  geo_longitude     object         
 15  temp5             object         
 16  temp4             object         
 17  temp3             object         
 18  

,id,id_equipo,id_tanque,fecha,hora,volumen,temperatura,codigo,baja,fuel_level_dmm,water_level_dmm,water_volume_lts,producto,geo_latitude,geo_longitude,temp5,temp4,temp3,temp2,temp1
0,60147,479,1037,2018-01-01,05:22:00,0032602.176,20.0,V0,0,,,,,None,None,,,,,
1,60148,479,1037,2018-01-01,06:35:00,0032515.134,21.0,V1,0,,,,,None,None,,,,,
2,60149,479,1037,2018-01-01,07:30:00,0032602.176,20.0,N0,0,,,,,None,None,,,,,
3,60150,479,1037,2018-01-01,08:07:00,0032576.632,19.0,V0,0,,,,,None,None,,,,,
4,60151,479,1037,2018-01-01,08:30:00,0032530.285,21.0,N1,0,,,,,None,None,,,,,


In [21]:
sis_historial_S1_2018.to_csv('data/sis_historial_S1_2018.csv', index=False)

In [24]:
%%time
%memit
tablas_H_S2_2018 = ['sis_historial_2018_7', 'sis_historial_2018_8', 'sis_historial_2018_9', 'sis_historial_2018_10', 'sis_historial_2018_11', 'sis_historial_2018_12']
sis_historial_S2_2018 = query_union_to_dataframe_with_chunk(tablas_H_S2_2018, conexion)
detalles_dataframe(sis_historial_S2_2018)

peak memory: 1440.36 MiB, increment: 0.22 MiB
Dimensión del dataset (filas, columnas): (6055448, 20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6055448 entries, 0 to 6055447
Data columns (total 20 columns):
 #   Column            Dtype          
---  ------            -----          
 0   id                int64          
 1   id_equipo         int64          
 2   id_tanque         int64          
 3   fecha             object         
 4   hora              timedelta64[ns]
 5   volumen           object         
 6   temperatura       object         
 7   codigo            object         
 8   baja              int64          
 9   fuel_level_dmm    object         
 10  water_level_dmm   object         
 11  water_volume_lts  object         
 12  producto          object         
 13  geo_latitude      object         
 14  geo_longitude     object         
 15  temp5             object         
 16  temp4             object         
 17  temp3             object         
 18  

,id,id_equipo,id_tanque,fecha,hora,volumen,temperatura,codigo,baja,fuel_level_dmm,water_level_dmm,water_volume_lts,producto,geo_latitude,geo_longitude,temp5,temp4,temp3,temp2,temp1
0,171296,162,198,2018-07-01,00:00:00,0001988.000,14.0,N0,0,,,,,None,None,,,,,
1,171297,162,190,2018-07-01,00:00:00,0016324.500,14.0,N0,0,,,,,None,None,,,,,
2,32883,421,933,2018-07-01,00:02:57,0001349.394,17.7,N2,0,00007661,00000015,0003.315,,None,None,,,,,
3,70602,374,874,2018-07-01,00:00:00,0013023.250,12.1,N1,0,00009991,00000000,00000000,,None,None,,,,,
4,70603,374,873,2018-07-01,00:00:00,0003690.240,15.7,N1,0,00003265,00000000,00000000,,None,None,,,,,


In [25]:
sis_historial_S2_2018.to_csv('data/sis_historial_S2_2018.csv', index=False)

In [26]:
%%time
%memit
tablas_H_T1_2019 = ['sis_historial_2019_1', 'sis_historial_2019_2', 'sis_historial_2019_3']
sis_historial_T1_2019 = query_union_to_dataframe_with_chunk(tablas_H_T1_2019, conexion)
detalles_dataframe(sis_historial_T1_2019)

peak memory: 3919.30 MiB, increment: 0.34 MiB
Dimensión del dataset (filas, columnas): (4523193, 20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4523193 entries, 0 to 4523192
Data columns (total 20 columns):
 #   Column            Dtype          
---  ------            -----          
 0   id                int64          
 1   id_equipo         int64          
 2   id_tanque         int64          
 3   fecha             object         
 4   hora              timedelta64[ns]
 5   volumen           object         
 6   temperatura       object         
 7   codigo            object         
 8   baja              int64          
 9   fuel_level_dmm    object         
 10  water_level_dmm   object         
 11  water_volume_lts  object         
 12  producto          object         
 13  geo_latitude      object         
 14  geo_longitude     object         
 15  temp5             object         
 16  temp4             object         
 17  temp3             object         
 18  

,id,id_equipo,id_tanque,fecha,hora,volumen,temperatura,codigo,baja,fuel_level_dmm,water_level_dmm,water_volume_lts,producto,geo_latitude,geo_longitude,temp5,temp4,temp3,temp2,temp1
0,513677,160,184,2019-01-01,0 days,0010717.040,24.3,N1,0,,,,2,None,None,,,,,
1,513678,160,185,2019-01-01,0 days,0004794.300,24.4,N1,0,,,,1,None,None,,,,,
2,513679,160,186,2019-01-01,0 days,0010848.520,25.3,N1,0,,,,0,None,None,,,,,
3,513680,160,1001,2019-01-01,0 days,0012460.722,26.0,N1,0,,,,3,None,None,,,,,
4,513681,160,180,2019-01-01,0 days,0006997.000,24.1,N1,0,,,,2,None,None,,,,,


In [27]:
sis_historial_T1_2019.to_csv('data/sis_historial_T1_2019.csv', index=False)

In [28]:
%%time
%memit
tablas_H_T2_2019 = ['sis_historial_2019_4', 'sis_historial_2019_5', 'sis_historial_2019_6']
for tabla in tablas_H_T2_2019:
    query = f'SELECT * FROM {tabla};'
    chunks=[]
    print(f'Leyendo...{tabla}')
    for chunk in pd.read_sql(query, conexion, chunksize = 10000):
        chunks.append(chunk)
    vars()[tabla] = chunks
print('Proceso finalizado')  

peak memory: 3512.58 MiB, increment: 0.08 MiB
Leyendo...sis_historial_2019_4
Leyendo...sis_historial_2019_5
Leyendo...sis_historial_2019_6
Proceso finalizado
Wall time: 9min 17s


In [29]:
sis_historial_T2_2019 = pd.concat(sis_historial_2019_4 + sis_historial_2019_5 + sis_historial_2019_6, ignore_index=True)
detalles_dataframe(sis_historial_T2_2019)

Dimensión del dataset (filas, columnas): (4856407, 20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4856407 entries, 0 to 4856406
Data columns (total 20 columns):
 #   Column            Dtype          
---  ------            -----          
 0   id                int64          
 1   id_equipo         int64          
 2   id_tanque         int64          
 3   fecha             object         
 4   hora              timedelta64[ns]
 5   volumen           object         
 6   temperatura       object         
 7   codigo            object         
 8   baja              int64          
 9   fuel_level_dmm    object         
 10  water_level_dmm   object         
 11  water_volume_lts  object         
 12  producto          object         
 13  geo_latitude      object         
 14  geo_longitude     object         
 15  temp5             object         
 16  temp4             object         
 17  temp3             object         
 18  temp2             object         
 19  temp1  

,id,id_equipo,id_tanque,fecha,hora,volumen,temperatura,codigo,baja,fuel_level_dmm,water_level_dmm,water_volume_lts,producto,geo_latitude,geo_longitude,temp5,temp4,temp3,temp2,temp1
0,97295,210,367,2019-04-01,00:00:00,0026423.000,20.2,N1,0,,,,0,None,None,,,,,
1,97296,210,367,2019-04-01,00:10:00,0026423.000,20.2,N1,0,,,,0,None,None,,,,,
2,420709,276,570,2019-04-01,00:00:00,0008346.200,22.9,N1,0,,,,0,None,None,,,,,
3,420710,276,571,2019-04-01,00:00:00,0008393.120,22.9,N1,0,,,,1,None,None,,,,,
4,420711,276,570,2019-04-01,00:01:00,0008346.200,22.9,N1,0,,,,0,None,None,,,,,


In [30]:
sis_historial_T2_2019.to_csv('data/sis_historial_T2_2019.csv', index=False)

In [37]:
%%time
%memit
tablas_H_T3_2019 = ['sis_historial_2019_7', 'sis_historial_2019_8', 'sis_historial_2019_9']
for tabla in tablas_H_T3_2019:
    query = f'SELECT * FROM {tabla};'
    chunks=[]
    print(f'Leyendo...{tabla}')
    for chunk in pd.read_sql(query, conexion, chunksize = 10000):
        chunks.append(chunk)
    vars()[tabla] = chunks
print('Proceso finalizado')  

peak memory: 1406.09 MiB, increment: 0.00 MiB
Leyendo...sis_historial_2019_7
Leyendo...sis_historial_2019_8
Leyendo...sis_historial_2019_9
Proceso finalizado
Wall time: 4min 7s


In [38]:
sis_historial_T3_2019 = pd.concat(sis_historial_2019_7 + sis_historial_2019_8 + sis_historial_2019_9, ignore_index=True)

In [40]:
%%time
%memit
tablas_H_T4_2019 = ['sis_historial_2019_10', 'sis_historial_2019_11', 'sis_historial_2019_12']
for tabla in tablas_H_T4_2019:
    query = f'SELECT * FROM {tabla};'
    chunks=[]
    print(f'Leyendo...{tabla}')
    for chunk in pd.read_sql(query, conexion, chunksize = 10000):
        chunks.append(chunk)
    vars()[tabla] = chunks
print('Proceso finalizado')  

peak memory: 4843.34 MiB, increment: -1.03 MiB
Leyendo...sis_historial_2019_10
Leyendo...sis_historial_2019_11
Leyendo...sis_historial_2019_12
Proceso finalizado
Wall time: 5min 33s


In [41]:
sis_historial_S2_2019 = pd.concat([sis_historial_T3_2019] + sis_historial_2019_10 + sis_historial_2019_11 + sis_historial_2019_12, ignore_index=True)

In [42]:
sis_historial_S2_2019.head()

,id,id_equipo,id_tanque,fecha,hora,volumen,temperatura,codigo,baja,fuel_level_dmm,water_level_dmm,water_volume_lts,producto,geo_latitude,geo_longitude,temp5,temp4,temp3,temp2,temp1
0,481478,477,1014,2019-07-01,00:00:43,0003452.000,15.5,N1,0,00004883,00000000,0000.000,3,None,None,,,,,
1,481479,477,1010,2019-07-01,00:01:36,0004009.250,16.5,N1,0,00005425,00000000,0000.000,0,None,None,,,,,
2,90688,421,933,2019-07-01,00:02:15,0001524.232,12.3,N2,0,00009099,00000019,0004.199,0,None,None,,,,,
3,90689,421,933,2019-07-01,00:12:03,0001524.232,12.3,N2,0,00009099,00000019,0004.199,0,None,None,,,,,
4,481480,477,1013,2019-07-01,00:02:17,0002144.700,16.5,N1,0,00003503,00000000,0000.000,2,None,None,,,,,


In [43]:
sis_historial_S2_2019.to_csv('data/sis_historial_S2_2019.csv', index=False)

In [ ]:
%%time
%memit
tablas_H_S1_2020 = ['sis_historial_2020_1', 'sis_historial_2020_2', 'sis_historial_2020_3', 'sis_historial_2020_4', 'sis_historial_2020_5']
for tabla in tablas_H_S1_2020:
    chunks=[]
    query = f'SELECT * FROM {tabla};'
    print(f'Leyendo...{tabla}')
    for chunk in pd.read_sql(query, conexion, chunksize = 10000):
        chunks.append(chunk)
    #vars()[tabla] = pd.concat(chunks, ignore_index=True)
    vars()[tabla] = chunks
print('Proceso finalizado')  

peak memory: 5512.91 MiB, increment: 0.06 MiB
Leyendo...sis_historial_2020_1


In [ ]:
sis_historial_T1_2020 = pd.concat(sis_historial_2020_1 + sis_historial_2020_2 + sis_historial_2020_3, ignore_index=True)
sis_historial_T2_2020 = pd.concat(sis_historial_2020_4 + sis_historial_2020_5, ignore_index=True)
sis_historial_S1_2020 = pd.concat([sis_historial_T1_2020, sis_historial_T2_2020], ignore_index=True)
sis_historial_S1_2020.head()

In [ ]:
sis_historial_S1_2020.to_csv('data/sis_historial_S1_2020.csv', index=False)

In [ ]:
# Close the session
mysqldb.close()

In [62]:
# Comprimidp en ZIP de archivos csv
import os
import zipfile
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))
            
zipf = zipfile.ZipFile('data/data_CTL.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('data/', zipf)
zipf.close()  # Importante hacerlo

## Datos públicos:
Bioetanol: producción y ventas (total país)
- Producción y ventas de bioetanol por mes en base a maíz y caña de azúcar (en metros cúbicos).
- Frecuencia de actualización: Mensualmente
- Último cambio: 15 de mayo de 2018

In [ ]:
url_bioetanol_nacional = 'http://datos.minem.gob.ar/dataset/5ce77ad1-c729-42cd-a8b5-2407de005e5b/resource/fd142c49-fa73-4e21-be1f-a10d6d67d05d/download/bioetanol-serie-produccion-y-ventas-total-pais.csv'
bioetanol_nacional=pd.read_csv(url_bioetanol_nacional)
detalles_dataframe(bioetanol_nacional)

Biodiesel: producción, ventas y exportaciones, total país
- Producción, ventas y exportaciones de biodiesel por mes en toneladas.
- Frecuencia de actualización: Mensualmente
- Último cambio: 15 de mayo de 2018

In [ ]:
url_biodiesel_nacional = 'http://datos.minem.gob.ar/dataset/5ce77ad1-c729-42cd-a8b5-2407de005e5b/resource/4e04bc74-8625-412c-acc2-48412f2509b4/download/biodiesel-serie-produccion-ventas-y-expo.csv'
# Leer CSV